In [215]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys  # Import Keys for special keys like Delete
from selenium.webdriver.chrome.service import Service
import time

# VARS
USERNAME = 'bh24.team4'
PASSWORD = 'traderion'
USER = 'Oscar Wooding'
ASSET_PATH = 'Equities/North America/NVDA US'
DIRECTION = 'Buy'  # or sell
UNITS = 1
REF = 'Automated Trading - Strategy: TEST   ' # must be at least 10 characters


In [216]:
print("Starting Web Application...")
# Set up the WebDriver (ensure you have the correct driver installed)
driver = webdriver.Chrome()

# Open the page
driver.get('https://portfolio.traderion.com/auth/login')  # Replace with the actual URL

# Wait for the page to load
driver.implicitly_wait(5)  # Wait for 10 seconds

try: 
    # Find username and password entry
    username_box = driver.find_element(By.ID, 'normal_login_username')  # Replace 'username' with the actual id
    password_box = driver.find_element(By.ID, 'normal_login_password')  # Replace 'password' with the actual id

    print("Logging in...")
    # Enter login credentials
    username_box.send_keys(USERNAME)  # Replace with your actual username
    password_box.send_keys(PASSWORD)  # Replace with your actual password
    # Find the login button and click it
    login_button = driver.find_element(By.CSS_SELECTOR, "button.ant-btn.login-form-button.ant-btn-primary")  # Replace 'login-button' with the actual id or name of the button
    login_button.click()

    # Locate the radio button corresponding to User using XPath
    radio = driver.find_element(By.XPATH, f"//label[span[text()='{USER}']]/span/input")
    radio.click()

    # Find the ok button and click it
    ok_button = driver.find_element(By.XPATH, "//button[span[text()='Ok']]")
    ok_button.click()

    print("Joining relevant server...")
    # Find the row with the text "Running" and locate the corresponding 'Join' link
    join_button = driver.find_element(By.XPATH, "//tr[td[text()='Running']]//a[text()='Join']")
    join_button.click()

    time.sleep(10)

    # switch to the latest window
    driver.switch_to.window(driver.window_handles[-1])
except Exception as e:
    print(f"An error occurred while logging in: {e}")


print("Selecting investment...")
try:
    driver.find_element(By.XPATH, f"//div[contains(span/text(), '{ASSET_PATH.split('/')[0]}')]").click()

    if len(ASSET_PATH.split('/')) == 3: 
        # Wait for the tab to be clickable and find the tab based on the value of asset country
        tab = driver.find_element(By.XPATH, f"//div[@role='tab' and text()='{ASSET_PATH.split('/')[1]}']")
        tab.click()
        time.sleep(2)

    elif len(ASSET_PATH.split('/')) == 2: 
        # Find asset and wait for the 'trade' button to be visible after expanding
        trade_button = driver.find_element(By.XPATH, f"//div[@role='tab' and text()='{ASSET_PATH.split('/')[1]}']//following-sibling::div[contains(@class, 'ant-collapse-item')]//button[contains(text(), 'Trade')]")
        trade_button.click()
        time.sleep(2)  # Adjust time as necessary

    # Locate the row based on the ticker 'XLE US'
    ticker = ASSET_PATH.split('/')[2]
    trade_button_xpath = f'//tr[td[contains(text(), "{ticker}")]]//button[span[text()="Trade"]]'

    # Find the "Trade" button and click it
    trade_button = driver.find_element(By.XPATH, trade_button_xpath)
    trade_button.click()
except Exception as e:
    print(f"An error occurred while selecting asset: {e}")


print("Entering trade...")
if DIRECTION == "Buy":
    radio_button_xpath = '//input[@value="0"]'  # XPath for Buy
elif DIRECTION == "Sell":
    radio_button_xpath = '//input[@value="1"]'  # XPath for Sell
else:
    radio_button_xpath = '//input[@value="2"]'  # XPath for Short Sell

# Select the order type radio button
try:
    order_type_radio = driver.find_element(By.XPATH, radio_button_xpath)
    order_type_radio.click()
    print(f"{DIRECTION} radio button selected successfully!")

    # Locate the quantity input box
    quantity_input_xpath = '//div[@class="ant-input-number-input-wrap"]/input[@class="ant-input-number-input"]'
    quantity_input = driver.find_element(By.XPATH, quantity_input_xpath)

    # Clear the existing quantity and enter the new value (1)
    quantity_input.click()
    quantity_input.send_keys(Keys.ARROW_LEFT)
    quantity_input.send_keys(Keys.DELETE)  # Press Delete key
    for _ in range(10):  # Ensure the field is cleared
        quantity_input.send_keys(Keys.ARROW_LEFT)
        quantity_input.send_keys(Keys.DELETE)

    quantity_input.send_keys(UNITS)  # Enter quantity of 1
    print("Quantity set to 1 successfully!")

    # Locate the investment rationale textarea
    rationale_textarea_xpath = '//textarea[@placeholder="Investment Rationale | required field min. 10 char."]'
    rationale_textarea = driver.find_element(By.XPATH, rationale_textarea_xpath)

    # Clear the existing rationale and enter the new value
    rationale_textarea.click()
    rationale_textarea.clear()  # Clear existing text
    rationale_textarea.send_keys(REF)  # Enter the investment rationale
    print("Investment rationale entered successfully!")

except Exception as e:
    print(f"An error occurred while entering trade details: {e}")


print("Placing trade...")
try: 
    # Locate the "Place Order" button
    place_order_button = driver.find_element(By.XPATH, "//button[span[text()='Place Order']]")  # Replace with the actual ID or locator

    # Click the "Place Order" button
    place_order_button.click()
    # Optional: Pause to see the result or confirmation
except Exception as e:
    print(f"An error occurred while executing trade: {e}")

time.sleep(5)
# Close the browser
driver.quit()

Starting Web Application...
Logging in...
Joining relevant server...
Selecting investment...
Entering trade...
Buy radio button selected successfully!
Quantity set to 1 successfully!
Investment rationale entered successfully!
Placing trade...
